In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import numpy.ma as ma
from mpl_toolkits.axes_grid1 import make_axes_locatable

import sys
sys.path.append('..')

import src as sc

# Validation

In [2]:
# Load data
hall_df, student_df, art_df = sc.load_data()

# Get art_capacity_df.
art_capacity_df = sc.get_art_capacity_with_downsampling(art_df,
                    categories = ["gender","race"])
# Get building_capacity_df.
building_capacity_df = sc.get_building_capacity_df()
building_capacity =  building_capacity_df.values
art_capacity = art_capacity_df["capacity"].values
# Fill buildings
sc.fill_buildings(student_df = student_df, hall_df = hall_df)

In [3]:
# Current assigment
current_assignment = pd.read_csv("current_assignment_df.csv", index_col = 0)
current_assignment = np.array(current_assignment)
current_assignment = current_assignment[:,0:11]

df_baseline = pd.DataFrame(index = ["Man","Non-Man","White","Non-White"])
df_optimized = pd.DataFrame(index = ["Man","Non-Man","White","Non-White"])

num_buildings =  building_capacity_df.shape[0]




    
# Get baseline stats.
#bs_gender = sc.baseline_average_value(category = "gender", in_group = "Man")
#bs_race = sc.baseline_average_value(category = "race", in_group = "White")
#df_baseline[i] = list(bs_gender) + list(bs_race)
# Find the appropriate scaling between the first and second terms of the optimization objective
# Fix alpha = -1
# Vary beta as follows: beta = [0.01 0.1 1 10 100 1000]
# For each beta, compute first term
# Compute second term without lambda
# See on average how large is the first term compared to the first
beta_values = np.array([0.01, 0.1, 1, 10,100,1000])
term1 = np.zeros((beta_values.shape[0],10))
term2 = np.zeros((beta_values.shape[0],10))
term3 = np.zeros((beta_values.shape[0],10))
for m in range(beta_values.shape[0]):
        # Compute full n_buildings x n_artworks cost matrix.
    cost_df = sc.compute_cost_matrix(art_df = art_df, 
                                    hall_df = hall_df,
                                    categories = ["gender","race"],
                                    alpha = -1,
                                    beta = beta_values[m])
    num_arts = cost_df.shape[1]
    t = np.arange(1,num_arts + 1)
    for k in range(1):
        # Pick a random permutation in S
        P = np.random.randn(num_buildings,num_arts)  
        # Projection
        for ii in range(num_buildings):
            v = P[ii,:]
            if np.all(v>0):
                v = v/np.sum(v)
                P[ii]=v * building_capacity[ii]
            else:
                mu = v[np.argsort(-v)]
                tmp = np.divide(np.cumsum(mu)-building_capacity[ii],t)
                idx_negative = np.argwhere(mu-tmp<=0)
                try:
                    idx_neg = idx_negative[0]
                    idx_neg = idx_neg.item()
                except:
                    idx_neg = -1
                theta = (np.sum(mu[0:idx_neg])-building_capacity[ii])/(idx_neg)
                P[ii,:] =  np.maximum(P[ii,:]-theta,0)  
        term1[m,k] = np.trace(np.matmul(np.transpose(cost_df.values),P))
        term2[m,k] = np.linalg.norm(np.sum(P,axis=0)-art_capacity)**2
        term3[m,k] = np.linalg.norm(P-current_assignment)**2
term1_avg = np.mean(term1)
term2_avg = np.mean(term2)
term3_avg = np.mean(term3)
lam_factor = term2_avg/term1_avg
tau_factor = term3_avg/term1_avg
print(term1_avg)
print(term2_avg)
print(term3_avg)

0.333765411835837
3174.0801556993197
3361.0272495949134


In [4]:
df_opt_dict = { }
for i in range(2):
    df_optimized = pd.DataFrame(index = ["Man","Non-Man","White","Non-White"])
    sc.fill_buildings(student_df = student_df, hall_df = hall_df)
    
    # Get baseline stats.
    bs_gender = sc.baseline_average_value(category = "gender", in_group = "Man")
    bs_race = sc.baseline_average_value(category = "race", in_group = "White")
    df_baseline[i] = list(bs_gender) + list(bs_race)
    beta_values = np.array([0.01, 0.1, 1, 10,100,1000])
    
    # Compute cost matrix
    cost_df = sc.compute_cost_matrix(art_df = art_df, 
                                    hall_df = hall_df,
                                    categories = ["gender","race"],
                                    alpha = -1,
                                    beta = beta_values[2])
    # Reduce cost df to remove duplicate columns.
    cost_df = cost_df.rename(columns = {art_capacity_df.loc[i,"tuple"]:art_capacity_df.loc[i,"string"] for i in art_capacity_df.index})
    # Optimize assignment_df
    #scaling_lam = np.array([0.01, 0.1, 1, 10,100])
    scaling_lam = np.array([0.0,0.00001 ])
    scaling_tau = np.array([0, 1.00])
    
    for p in scaling_lam:
        for q in scaling_tau:
            assignment_df = sc.learn_optimal_assignment(cost_df, 
                                 building_capacity_df, 
                                 art_capacity_df, current_assignment, 
                                 lam = lam_factor*p, tau=tau_factor*q, init=1) # lambda, you can change this.
    
            # Get optimized stats.
            os_gender = sc.optimized_average_value(assignment_df.copy(), category = "gender", in_group = "Man")
            os_race = sc.optimized_average_value(assignment_df.copy(), category = "race", in_group = "White")
            df_optimized[f"{p}_{q}"] = list(os_gender) + list(os_race)
    df_opt_dict[i] = df_optimized.copy()


    
#df_val = pd.DataFrame(index = df_optimized.index)
#df_val["baseline_mean"] = df_baseline.mean(axis = 1)
#df_val["baseline_95_CI"] = .975 * np.std(df_baseline, axis =1) / df_optimized.shape[1]
#df_val["optimized_mean"] = df_optimized.mean(axis = 1)
#df_val["optimized_95_CI"] = .975 * np.std(df_optimized, axis =1) / df_optimized.shape[1]
#df_val

In [5]:
df_optimized


,0.0_0.0,0.0_1.0,1e-05_0.0,1e-05_1.0
Man,0.039045,2.993557,0.264202,2.988027
Non-Man,0.090719,11.729602,0.034707,11.735362
White,0.000000,12.832021,0.129644,12.823045
Non-White,0.153998,2.031100,0.142729,2.041863


In [6]:
df_val

NameError: name 'df_val' is not defined

In [7]:
df_opt_dict

{0:             0.0_0.0    0.0_1.0  1e-05_0.0  1e-05_1.0
 Man        0.039002   2.951038   0.269928   2.945508
 Non-Man    0.091223  11.915734   0.031655  11.921496
 White      0.000000  12.796695   0.122568  12.787682
 Non-White  0.153539   2.043588   0.148942   2.054297,
 1:             0.0_0.0    0.0_1.0  1e-05_0.0  1e-05_1.0
 Man        0.039045   2.993557   0.264202   2.988027
 Non-Man    0.090719  11.729602   0.034707  11.735362
 White      0.000000  12.832021   0.129644  12.823045
 Non-White  0.153998   2.031100   0.142729   2.041863}